Due to very low volume of data witch claim frequency above 0, I will use two different aproaches to modeling this subject:
1. GLM, zero-inflated model - the key point in this aproach is to choose proper distribution function for our variable. Due to high skewness and variance aproximatelly 2 times higher than mean I will compere Poisson and Negative Binomial distribution and choose which will better suit the data.
2. two level model - classification + regression

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

### Data transformation

In [2]:
df = pd.read_csv("feature_selected_data.csv")
df

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,Area,VehBrand,VehGas,Region,ClaimFrequency,VehPowerDriverAge,TransformBonusMalus,TransformDensity
0,1.0,0.10000,5.0,0.0,55.0,D,B12,Regular,R82,1.000000,0.090909,3.912023,7.104144
1,1.0,0.77000,5.0,0.0,55.0,D,B12,Regular,R82,1.298701,0.090909,3.912023,7.104144
2,1.0,0.75000,6.0,2.0,52.0,B,B12,Diesel,R22,1.333333,0.115385,3.912023,3.988984
3,1.0,0.84000,7.0,0.0,46.0,B,B12,Diesel,R72,1.190476,0.152174,3.912023,4.330733
4,1.0,0.52000,6.0,2.0,38.0,E,B12,Regular,R31,1.923077,0.157895,3.912023,8.007367
...,...,...,...,...,...,...,...,...,...,...,...,...,...
677997,0.0,0.00274,4.0,0.0,54.0,E,B12,Regular,R93,0.000000,0.074074,3.912023,8.106816
677998,0.0,0.00274,4.0,0.0,41.0,E,B12,Regular,R11,0.000000,0.097561,4.553877,9.195227
677999,0.0,0.00274,6.0,2.0,45.0,D,B12,Diesel,R82,0.000000,0.133333,3.912023,7.187657
678000,0.0,0.00274,4.0,0.0,60.0,B,B12,Regular,R26,0.000000,0.066667,3.912023,4.553877


In [ ]:
to_encode = df.select_dtypes(include=object)

ohe = OneHotEncoder(sparse_output=False, drop='first') # Area_A, Region_R11, VehBrand_B1, VehGas_Diesel

encoded_array = ohe.fit_transform(to_encode)
encoded_df = pd.DataFrame(encoded_array, columns=ohe.get_feature_names_out(to_encode.columns))
df_final = pd.concat([df.select_dtypes(include=np.number), encoded_df], axis=1)
df_final

In [5]:
train, test = train_test_split(df_final, test_size=0.2, stratify=df_final["isClaim"], random_state=42)

In [6]:
train_data_glm = train.drop(["isClaim"], axis=1)
formula = "ClaimFrequency ~ " + " + ".join(train_data_glm.columns.difference(["ClaimFrequency"]))

model_pois = smf.glm(formula, 
                      data=train_data_glm, 
                      family=sm.families.Poisson()).fit()

model_nb = smf.glm(formula, 
                    data=train_data_glm, 
                    family=sm.families.NegativeBinomial()).fit()

lr_stat = 2 * (model_nb.llf - model_pois.llf) 
p_value = stats.chi2.sf(lr_stat, df=1) 

print(f"LR Statistic: {lr_stat}, p-value: {p_value}")

d:\Projects\PricingRecrutationProject\FrequencyOfClaimsPrediction\.venv\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


LR Statistic: 20492.93491096847, p-value: 0.0


Based on the likelihood ratio test we can see that Negative Binomial model significantly improves the fit and is worth using instead of Poisson.

In [7]:
model_nb.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         ClaimFrequency   No. Observations:               542401
Model:                            GLM   Df Residuals:                   542357
Model Family:        NegativeBinomial   Df Model:                           43
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4709e+05
Date:                Mon, 10 Mar 2025   Deviance:                   2.0552e+05
Time:                        17:41:14   Pearson chi2:                 9.58e+05
No. Iterations:                     6   Pseudo R-squ. (CS):           0.008697
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -8.7076      0.119    -73.299      0.000      -8.940      -8.475
Area_B                  0.0122      0.025      0.490      0.624      -0.037       0.061
Area_C                 -0.0117      0.032     -0.361      0.718      -0.075       0.052
Area_D                  0.0270      0.049      0.552      0.581      -0.069       0.123
Area_E                  0.0104      0.065      0.161      0.872      -0.117       0.138
Area_F                  0.0041      0.089      0.046      0.963      -0.170       0.178
DrivAge                 0.0111      0.001     14.199      0.000       0.010       0.013
Region_R21              0.0148      0.078      0.190      0.849      -0.138       0.168
Region_R22              0.0280      0.050      0.563      0.573      -0.069       0.126
Region_R23             -0.3338      0.056     -5.930      0.000      -0.444      -0.223
Region_R24              0.1657      0.023      7.086      0.000       0.120       0.212
Region_R25              0.0817      0.045      1.826      0.068      -0.006       0.169
Region_R26             -0.0599      0.047     -1.271      0.204      -0.152       0.032
Region_R31             -0.1980      0.033     -5.983      0.000      -0.263      -0.133
Region_R41             -0.1149      0.044     -2.597      0.009      -0.202      -0.028
Region_R42              0.0407      0.087      0.466      0.641      -0.130       0.212
Region_R43             -0.0575      0.120     -0.477      0.633      -0.294       0.179
Region_R52              0.0183      0.029      0.624      0.532      -0.039       0.076
Region_R53              0.2127      0.028      7.570      0.000       0.158       0.268
Region_R54              0.0384      0.037      1.032      0.302      -0.035       0.111
Region_R72             -0.1257      0.032     -3.976      0.000      -0.188      -0.064
Region_R73             -0.2665      0.041     -6.474      0.000      -0.347      -0.186
Region_R74              0.2614      0.062      4.193      0.000       0.139       0.384
Region_R82              0.1099      0.023      4.768      0.000       0.065       0.155
Region_R83             -0.2363      0.068     -3.454      0.001      -0.370      -0.102
Region_R91             -0.1556      0.031     -5.032      0.000      -0.216      -0.095
Region_R93             -0.0473      0.024     -1.995      0.046      -0.094      -0.001
Region_R94              0.1339      0.060      2.216      0.027       0.015       0.252
TransformBonusMalus     1.3423      0.025     54.499      0.000       1.294       1.391
TransformDensity        0.0305      0.012      2.478      0.013       0.006       0.055
VehAge                 -0.0307      0.001    -26.871      0.000      -0.033      -0.028
VehBrand_B10           -0.0727      0.036     -2.013      0.044      -0.144      -0.002
Veh

As we can see many of the given feature doesn't seem to be statistically significant. Therefore I will use backward step method for future selection

In [8]:
def backward_stepwise_selection(data, target, initial_features, family, significance_level=0.05):
    """
    Perform backward stepwise selection for GLM.
    
    Parameters:
        data (pd.DataFrame): The dataset.
        target (str): The dependent variable.
        initial_features (list): List of all features to start with.
        significance_level (float): P-value threshold for feature removal.

    Returns:
        str: Final GLM formula with selected features.
    """
    selected_features = initial_features.copy()
    
    while len(selected_features) > 0:
        # Build formula
        formula = f"{target} ~ " + " + ".join(selected_features)
        
        # Fit the model
        model = smf.glm(formula, data=data, family=family).fit()
        
        # Get p-values
        p_values = model.pvalues.drop("Intercept", errors="ignore")
        
        # Find the feature with the highest p-value
        max_p_value = p_values.max()
        worst_feature = p_values.idxmax()

        print(f"Max p-value: {max_p_value:.4f} (Feature: {worst_feature})")
        
        # Stop if all remaining features are significant
        if max_p_value < significance_level:
            break

        # Remove the least significant feature
        selected_features.remove(worst_feature)
        print(f"Removing feature: {worst_feature}")

    # Return final selected model
    return model

In [9]:
model_nb_selected = \
    backward_stepwise_selection(data=train_data_glm, 
                            target="ClaimFrequency", 
                            initial_features=list(train_data_glm.columns.difference(["ClaimFrequency"])),
                            family=sm.families.NegativeBinomial())

d:\Projects\PricingRecrutationProject\FrequencyOfClaimsPrediction\.venv\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


Max p-value: 0.9631 (Feature: Area_F)
Removing feature: Area_F
Max p-value: 0.8505 (Feature: Region_R21)
Removing feature: Region_R21
Max p-value: 0.8354 (Feature: VehBrand_B3)
Removing feature: VehBrand_B3
Max p-value: 0.8044 (Feature: VehPower)
Removing feature: VehPower
Max p-value: 0.7428 (Feature: Area_E)
Removing feature: Area_E
Max p-value: 0.6521 (Feature: VehBrand_B11)
Removing feature: VehBrand_B11
Max p-value: 0.6401 (Feature: Region_R43)
Removing feature: Region_R43
Max p-value: 0.6238 (Feature: Area_B)
Removing feature: Area_B
Max p-value: 0.6127 (Feature: Region_R42)
Removing feature: Region_R42
Max p-value: 0.5384 (Feature: Region_R22)
Removing feature: Region_R22
Max p-value: 0.5437 (Feature: Region_R52)
Removing feature: Region_R52
Max p-value: 0.4270 (Feature: VehBrand_B13)
Removing feature: VehBrand_B13
Max p-value: 0.3782 (Feature: Region_R54)
Removing feature: Region_R54
Max p-value: 0.2342 (Feature: VehBrand_B6)
Removing feature: VehBrand_B6
Max p-value: 0.2378 (F

In [10]:
model_nb_selected.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         ClaimFrequency   No. Observations:               542401
Model:                            GLM   Df Residuals:                   542377
Model Family:        NegativeBinomial   Df Model:                           23
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4710e+05
Date:                Mon, 10 Mar 2025   Deviance:                   2.0554e+05
Time:                        17:45:49   Pearson chi2:                 9.58e+05
No. Iterations:                     6   Pseudo R-squ. (CS):           0.008664
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -8.7049      0.110    -79.178      0.000      -8.920      -8.489
Area_D                  0.0276      0.013      2.188      0.029       0.003       0.052
DrivAge                 0.0109      0.001     21.594      0.000       0.010       0.012
Region_R23             -0.3440      0.054     -6.350      0.000      -0.450      -0.238
Region_R24              0.1534      0.016      9.612      0.000       0.122       0.185
Region_R31             -0.2107      0.029     -7.147      0.000      -0.268      -0.153
Region_R41             -0.1300      0.041     -3.143      0.002      -0.211      -0.049
Region_R53              0.1983      0.022      8.825      0.000       0.154       0.242
Region_R72             -0.1383      0.028     -5.001      0.000      -0.193      -0.084
Region_R73             -0.2795      0.038     -7.296      0.000      -0.355      -0.204
Region_R74              0.2462      0.060      4.090      0.000       0.128       0.364
Region_R82              0.0979      0.017      5.674      0.000       0.064       0.132
Region_R83             -0.2496      0.067     -3.744      0.000      -0.380      -0.119
Region_R91             -0.1728      0.026     -6.534      0.000      -0.225      -0.121
Region_R93             -0.0595      0.018     -3.240      0.001      -0.096      -0.024
Region_R94              0.1189      0.058      2.038      0.042       0.005       0.233
TransformBonusMalus     1.3434      0.024     56.969      0.000       1.297       1.390
TransformDensity        0.0310      0.003      9.921      0.000       0.025       0.037
VehAge                 -0.0303      0.001    -29.416      0.000      -0.032      -0.028
VehBrand_B14           -0.2357      0.078     -3.026      0.002      -0.388      -0.083
VehBrand_B4            -0.0672      0.028     -2.401      0.016      -0.122      -0.012
VehBrand_B5             0.0853      0.023      3.735      0.000       0.041       0.130
VehGas_Regular          0.0945      0.011      8.992      0.000       0.074       0.115
VehPowerDriverAge       0.4760      0.095      5.023      0.000       0.290       0.662
=======================================================================================
"""

In [11]:
model_nb_selected.save("model_nb_selected.pickle")

In [ ]:
# TODO: zbadać gdzie popełnia błędy, RMSE, przeuczenie